In [1]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import QED, Descriptors
import warnings
warnings.filterwarnings('ignore')

# Load the dataset
df = pd.read_csv('robin_clean.csv')

print(f"Dataset loaded: {len(df)} molecules")
print(f"Columns: {df.columns.tolist()}")
df.head()


Dataset loaded: 24501 molecules
Columns: ['Name', 'Smile', 'TPP', 'Glutamine_RS', 'ZTP', 'SAM_ll', 'PreQ1']


,Name,Smile,TPP,Glutamine_RS,ZTP,SAM_ll,PreQ1
0,0054-0090,CC(c1ccc(c(c1)N)[Br])=O,0,0.0,1.0,0.0,0
1,0061-0013,c1cc2c(cc1N)non2,0,0.0,0.0,0.0,0
2,0062-0039,C(n1cncn1)O,0,0.0,0.0,0.0,0
3,0082-0017,c1ccc2c(c1)ccc1c2nc(N)s1,0,0.0,0.0,0.0,0
4,0083-0114,CCN(CC)c1ccc2C(C)=C(CO)C(=O)Oc2c1,0,0.0,0.0,0.0,0


In [2]:
# Calculate QED and SA scores for all molecules
def calculate_molecular_properties(smiles):
    """Calculate QED and SA score approximation for a SMILES string."""
    try:
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return None, None
        
        # Calculate QED (Drug-likeness)
        qed_value = QED.qed(mol)
        
        # Calculate SA Score approximation
        # (same formula used in server.py)
        num_rings = Descriptors.RingCount(mol)
        num_heteroatoms = Descriptors.NumHeteroatoms(mol)
        num_rotatable_bonds = Descriptors.NumRotatableBonds(mol)
        mol_weight = Descriptors.MolWt(mol)
        
        sa_score = 1 + (num_rings * 0.5) + (num_heteroatoms * 0.3) + \
                   (num_rotatable_bonds * 0.2) + (mol_weight / 100)
        sa_score = min(10, max(1, sa_score))
        
        return qed_value, sa_score
    except Exception as e:
        print(f"Error processing SMILES {smiles}: {e}")
        return None, None

# Calculate properties for all molecules
print("Calculating molecular properties (this may take a minute)...")
properties = df['Smile'].apply(calculate_molecular_properties)
df['QED'] = properties.apply(lambda x: x[0] if x else None)
df['SA_Score'] = properties.apply(lambda x: x[1] if x else None)

print(f"Properties calculated for {df['QED'].notna().sum()} molecules")
print(f"Missing values: {df['QED'].isna().sum()}")
df[['Name', 'Smile', 'QED', 'SA_Score']].head(10)


Calculating molecular properties (this may take a minute)...


[22:29:15] Conflicting single bond directions around double bond at index 27.
[22:29:15]   BondStereo set to STEREONONE and single bond directions set to NONE.


Properties calculated for 24501 molecules
Missing values: 0


,Name,Smile,QED,SA_Score
0,0054-0090,CC(c1ccc(c(c1)N)[Br])=O,0.574900,4.74062
1,0061-0013,c1cc2c(cc1N)non2,0.542521,4.55126
2,0062-0039,C(n1cncn1)O,0.502522,3.89093
3,0082-0017,c1ccc2c(c1)ccc1c2nc(N)s1,0.605445,5.40266
4,0083-0114,CCN(CC)c1ccc2C(C)=C(CO)C(=O)Oc2c1,0.858839,6.61321
5,0086-0080,c1ccc(cc1)NC1C(N(C(=O)S1)c1ccccc1)=O,0.938961,7.44340
6,0092-0008,CNc1ccc(c2C(c3ccccc3C(c12)=O)=O)[Br],0.749378,7.06154
7,0096-0280,C(CO)NC(C1=Cc2ccccc2OC1=O)=O,0.757097,6.43223
8,0098-0003,CCCCCCOC(C(CC(C)C)N)=O,0.499682,5.65337
9,0099-0265,C1=C(c2ccc(cc2OC1=O)N)C(F)(F)F,0.557010,6.09157


In [3]:
# Define the targets
targets = ['TPP', 'Glutamine_RS', 'ZTP', 'SAM_ll', 'PreQ1']

# Create summary statistics
summary_data = []

for target in targets:
    # Total tested
    total_tested = len(df)
    
    # Number of hits (where target column == 1)
    hits = df[df[target] == 1]
    num_hits = len(hits)
    
    # Hit rate (percentage)
    hit_rate = (num_hits / total_tested) * 100
    
    # Mean QED for hits only (excluding NaN values)
    mean_qed = hits['QED'].mean()
    
    # Mean SA Score for hits only (excluding NaN values)
    mean_sa = hits['SA_Score'].mean()
    
    summary_data.append({
        'Target': target,
        'Total Tested': total_tested,
        '# Hits': num_hits,
        'Hit Rate': f'{hit_rate:.2f}%',
        'Mean QED (hits)': f'{mean_qed:.3f}' if not pd.isna(mean_qed) else 'N/A',
        'Mean SA (hits)': f'{mean_sa:.2f}' if not pd.isna(mean_sa) else 'N/A'
    })

# Create DataFrame
summary_df = pd.DataFrame(summary_data)

print("\n" + "="*80)
print("SUMMARY STATISTICS - ROBIN Screening Dataset")
print("="*80)
print(summary_df.to_string(index=False))
print("="*80)



SUMMARY STATISTICS - ROBIN Screening Dataset
      Target  Total Tested  # Hits Hit Rate Mean QED (hits) Mean SA (hits)
         TPP         24501     162    0.66%           0.639           8.47
Glutamine_RS         24501      70    0.29%           0.684           7.55
         ZTP         24501     170    0.69%           0.657           8.46
      SAM_ll         24501     196    0.80%           0.641           8.24
       PreQ1         24501     166    0.68%           0.679           8.19


In [4]:
# Display the table with better formatting
from IPython.display import display, HTML

# Style the dataframe for better display
styled_summary = summary_df.style.set_properties(**{
    'text-align': 'left',
    'font-size': '12pt',
    'border': '1px solid black'
}).set_table_styles([
    {'selector': 'th', 'props': [('background-color', '#2196F3'), 
                                   ('color', 'white'),
                                   ('font-weight', 'bold'),
                                   ('text-align', 'center'),
                                   ('padding', '10px')]},
    {'selector': 'td', 'props': [('padding', '8px'),
                                   ('text-align', 'center')]}
])

display(styled_summary)

# Optionally save to CSV
summary_df.to_csv('screening_summary_statistics.csv', index=False)
print("\n✓ Summary table saved to 'screening_summary_statistics.csv'")


,Target,Total Tested,# Hits,Hit Rate,Mean QED (hits),Mean SA (hits)
0,TPP,24501,162,0.66%,0.639,8.47
1,Glutamine_RS,24501,70,0.29%,0.684,7.55
2,ZTP,24501,170,0.69%,0.657,8.46
3,SAM_ll,24501,196,0.80%,0.641,8.24
4,PreQ1,24501,166,0.68%,0.679,8.19



✓ Summary table saved to 'screening_summary_statistics.csv'


In [5]:
# Additional overall statistics
print("\n" + "="*80)
print("ADDITIONAL DATASET STATISTICS")
print("="*80)

# Total unique hits across all targets
unique_hits = df[(df['TPP'] == 1) | (df['Glutamine_RS'] == 1) | 
                 (df['ZTP'] == 1) | (df['SAM_ll'] == 1) | (df['PreQ1'] == 1)]
print(f"Total unique hits (any target): {len(unique_hits)}")

# Inactive molecules (no activity against any target)
inactive = df[(df['TPP'] == 0) & (df['Glutamine_RS'] == 0) & 
              (df['ZTP'] == 0) & (df['SAM_ll'] == 0) & (df['PreQ1'] == 0)]
print(f"Inactive molecules (all targets): {len(inactive)}")

# Promiscuous binders (active against multiple targets)
df['num_targets_active'] = df[targets].sum(axis=1)
promiscuous = df[df['num_targets_active'] > 1]
print(f"Promiscuous binders (>1 target): {len(promiscuous)}")

# Overall property statistics
print("\n" + "-"*80)
print("MOLECULAR PROPERTIES - ALL MOLECULES")
print("-"*80)
print(f"Mean QED (all):       {df['QED'].mean():.3f} ± {df['QED'].std():.3f}")
print(f"Mean SA Score (all):  {df['SA_Score'].mean():.2f} ± {df['SA_Score'].std():.2f}")

print("\n" + "-"*80)
print("MOLECULAR PROPERTIES - ACTIVE HITS (ANY TARGET)")
print("-"*80)
print(f"Mean QED (hits):      {unique_hits['QED'].mean():.3f} ± {unique_hits['QED'].std():.3f}")
print(f"Mean SA Score (hits): {unique_hits['SA_Score'].mean():.2f} ± {unique_hits['SA_Score'].std():.2f}")
print("="*80)



ADDITIONAL DATASET STATISTICS
Total unique hits (any target): 553
Inactive molecules (all targets): 3742
Promiscuous binders (>1 target): 143

--------------------------------------------------------------------------------
MOLECULAR PROPERTIES - ALL MOLECULES
--------------------------------------------------------------------------------
Mean QED (all):       0.705 ± 0.150
Mean SA Score (all):  8.50 ± 1.32

--------------------------------------------------------------------------------
MOLECULAR PROPERTIES - ACTIVE HITS (ANY TARGET)
--------------------------------------------------------------------------------
Mean QED (hits):      0.664 ± 0.165
Mean SA Score (hits): 8.28 ± 1.47
